## 1. Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
pd.set_option('display.max_rows', 1000); pd.set_option('display.max_columns', 1000); pd.set_option('display.width', 1000)

## 2. Functions

In [3]:
# Flag anomalies with confidence level 
def calcualte_loss (predictions, pred_status, y_test, confidence_level):
    max_prob = np.max(predictions, axis=1)
    anomalies = (pred_status != y_test) & (max_prob > confidence_level)
    indexes = np.where(anomalies)
    return indexes

## 3. Data Preprocessing

### a. Load Datasets

In [4]:
# Load dataset
tmp = pd.read_csv("hisevents_nov_2023.csv")

C:\Users\Chun Quan\AppData\Local\Temp\ipykernel_17508\3670644652.py:2: DtypeWarning: Columns (21,22,23,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv("hisevents_nov_2023.csv")


The warning indicates that columns (21, 22, 23, 24, 25, and 28) contain mixed data types. We should examine the unique values in these columns to better understand the data types and update them accordingly.

In [5]:
# Examine the unique values in the col
tmp.iloc[:,28].unique()

array([nan, 'C760', 'C751A'], dtype=object)

Since all of these columns are in string format, we will update them to have the str datatype.

In [6]:
# Update col datatype in df
tmp.iloc[:,21]=tmp.iloc[:,21].astype(str)
tmp.iloc[:,22]=tmp.iloc[:,22].astype(str)
tmp.iloc[:,23]=tmp.iloc[:,23].astype(str)
tmp.iloc[:,24]=tmp.iloc[:,24].astype(str)
tmp.iloc[:,25]=tmp.iloc[:,25].astype(str)
tmp.iloc[:,28]=tmp.iloc[:,28].astype(str)

### b. Extract Escalator Dataset 

In [7]:
# Filter MESSAGE that contains Escalator: Operating Status, ASSETNAME that contains L&E/.../../E..
tmp = tmp[(tmp['MESSAGE']=='Escalator: Operating Status') & (tmp['ASSETNAME'].str.match(r'L&E/[A-Z]{3}/\w{1}\d{1}/E\d{2}'))][['ASSETNAME','STATUS','dttm']].reset_index(drop=True)
tmp['dttm']=pd.to_datetime(tmp['dttm'])

### c. EDA

In [8]:
# Check for Missing Data
tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57699 entries, 0 to 57698
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ASSETNAME  57699 non-null  object        
 1   STATUS     57699 non-null  object        
 2   dttm       57699 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 1.3+ MB


In [9]:
tmp.sample(3)

,ASSETNAME,STATUS,dttm
25963,L&E/WLH/B1/E04,STOP,2023-11-15 05:17:46
16948,L&E/BNK/B1/E06,DOWN,2023-11-10 05:51:38
2898,L&E/SKG/O2/E01,UP,2023-11-02 12:25:55


### d. Group by 30 Minute Frequency

In [10]:
# Reshape to 30 mintue frequency, count status
grouped_30 = tmp.groupby(
    ['ASSETNAME', pd.Grouper(
        key='dttm', freq='30T')]).apply(lambda x: pd.Series({
   'status_count': x['STATUS'].isin(['STOP', 'UP', 'DOWN']).sum(),
   'STATUS': x['STATUS'].tolist()
})).reset_index()

In [11]:
# Convert to datetime format, create a col for date and time
grouped_30['dttm']=pd.to_datetime(grouped_30['dttm'])
grouped_30['date']=grouped_30['dttm'].dt.date
grouped_30['time']=grouped_30['dttm'].dt.time

In [12]:
grouped_30.sample(3)

,ASSETNAME,dttm,status_count,STATUS,date,time
30021,L&E/WLH/B2/E01,2023-11-21 04:00:00,1,[DOWN],2023-11-21,04:00:00
10277,L&E/DBG/B3/E04,2023-11-15 00:30:00,1,[STOP],2023-11-15,00:30:00
19413,L&E/OTP/B1/E08,2023-11-01 00:00:00,1,[STOP],2023-11-01,00:00:00


In [13]:
# Keep the last index of STATUS for length of status > 1
grouped = grouped_30.copy(deep=True)
for index, row in grouped.iterrows():
    if row['status_count'] > 1:
        grouped.loc[index,'STATUS'] = row['STATUS'][-1]

grouped.drop(columns='status_count', inplace=True)

# Remove list datatype in STATUS col
grouped= grouped.explode('STATUS')

In [14]:
grouped.sample(3)

,ASSETNAME,dttm,STATUS,date,time
12891,L&E/FRP/B2/E01,2023-11-23 00:00:00,STOP,2023-11-23,00:00:00
18698,L&E/OTP/B1/E02,2023-11-01 00:00:00,STOP,2023-11-01,00:00:00
780,L&E/BGK/B1/E02,2023-11-09 11:00:00,UP,2023-11-09,11:00:00


In [15]:
# Extend data to include missing 30 minute interval, fill in missing values
tmp_grouped = grouped.groupby('ASSETNAME')

tmp_list = []
for index, data in tmp_grouped:
   grouped_df = data.set_index('dttm').resample('30T').ffill()
   grouped_df = grouped_df.reset_index()
   grouped_df['time'] = grouped_df['dttm'].dt.time
   grouped_df['date'] = grouped_df['dttm'].dt.date
   
   grouped_df  = grouped_df.sort_values(by="dttm", ascending=True)

   next_status = [
      grouped_df.iloc[x+1]["STATUS"] for x in range(len(grouped_df)-1)]
   next_status += [None] # Last value in list set to None
   
   # Append the grouped dataframes
   grouped_df["next_status"] = next_status
   tmp_list.append(grouped_df.iloc[:-1]) # Except last row

# Concatenate the dataframes
grouped_df = pd.concat(tmp_list).reset_index(drop=True)

In [16]:
grouped_df.head(3)

,dttm,ASSETNAME,STATUS,date,time,next_status
0,2023-11-01 00:00:00,L&E/BGK/B1/E01,STOP,2023-11-01,00:00:00,STOP
1,2023-11-01 00:30:00,L&E/BGK/B1/E01,STOP,2023-11-01,00:30:00,STOP
2,2023-11-01 01:00:00,L&E/BGK/B1/E01,STOP,2023-11-01,01:00:00,STOP


## 4. Train Test Split

Training data: all except 01-12-2023 data

In [17]:
train_data= grouped_df[pd.to_datetime(grouped_df['date']) != '2023-12-01'].reset_index(drop=True)

In [18]:
train_data.head(3)

,dttm,ASSETNAME,STATUS,date,time,next_status
0,2023-11-01 00:00:00,L&E/BGK/B1/E01,STOP,2023-11-01,00:00:00,STOP
1,2023-11-01 00:30:00,L&E/BGK/B1/E01,STOP,2023-11-01,00:30:00,STOP
2,2023-11-01 01:00:00,L&E/BGK/B1/E01,STOP,2023-11-01,01:00:00,STOP


Testing data: 01-12-2023 data

In [19]:
test_data = grouped_df[pd.to_datetime(grouped_df['date']) == pd.to_datetime('2023-12-01')].reset_index(drop=True)

In [20]:
test_data.head(3)

,dttm,ASSETNAME,STATUS,date,time,next_status
0,2023-12-01 00:00:00,L&E/BGK/B1/E01,UP,2023-12-01,00:00:00,STOP
1,2023-12-01 00:30:00,L&E/BGK/B1/E01,STOP,2023-12-01,00:30:00,STOP
2,2023-12-01 01:00:00,L&E/BGK/B1/E01,STOP,2023-12-01,01:00:00,STOP


## 5. Data Preprocessing for LSTM/ SimpleRNN

In [21]:
# Label Encoding
status_to_index = {'STOP': 0,'UP': 1,'DOWN': 2}

train_data["STATUS"] = [
    status_to_index[x] for x in list(train_data["STATUS"].values)]

test_data["STATUS"] = [
    status_to_index[x] for x in list(test_data["STATUS"].values)]

train_data["next_status"] = [
    status_to_index[x] for x in list(train_data["next_status"].values)]

test_data["next_status"] = [
    status_to_index[x] for x in list(test_data["next_status"].values)]

# Time Encoding
time_interval = list(sorted(list(pd.unique(train_data["time"]))))
time_to_index = dict([
    (time_interval[x], x) for x in range(len(time_interval))])

# Each time is assigned to an index
train_data["time_index"] = [
    time_to_index[x] for x in list(train_data["time"].values)]
test_data["time_index"] = [
    time_to_index[x] for x in list(test_data["time"].values)]

# Escalator Encoding
asset_names  = list(
    sorted(list(pd.unique(train_data["ASSETNAME"]))))
esc_to_index = dict([
    (asset_names[x], x) for x in range(len(asset_names))])

# Each assetname is assgined to an index
train_data["esc_index"] = [
    esc_to_index[x] for x in list(train_data["ASSETNAME"].values)]
test_data["esc_index"] = [
    esc_to_index[x] for x in list(test_data["ASSETNAME"].values)]

In [22]:
train_data.sample(3)

,dttm,ASSETNAME,STATUS,date,time,next_status,time_index,esc_index
298738,2023-11-11 05:30:00,L&E/WLH/B2/E02,2,2023-11-11,05:30:00,2,11,208
193105,2023-11-05 08:00:00,L&E/OTP/B1/E02,2,2023-11-05,08:00:00,2,16,134
14927,2023-11-12 00:30:00,L&E/BNK/B1/E04,0,2023-11-12,00:30:00,0,1,10


In [23]:
test_data.sample(3)

,dttm,ASSETNAME,STATUS,date,time,next_status,time_index,esc_index
376,2023-12-01 03:30:00,L&E/BNK/B2/E01,0,2023-12-01,03:30:00,0,7,13
990,2023-12-01 05:30:00,L&E/DBG/B2/E09,1,2023-12-01,05:30:00,1,11,55
12,2023-12-01 06:00:00,L&E/BGK/B1/E01,1,2023-12-01,06:00:00,1,12,0


## 6. LSTM

In [24]:
def build_model():
    x_input = tf.keras.Input( 
        shape=(3,1,), dtype="int32", name="x_input")
    x_status = tf.keras.layers.Embedding(3, 32)(x_input[:, 1, :])
    x_tm_day = tf.keras.layers.Embedding(48, 32)(x_input[:, 0, :])
    x_asset  = tf.keras.layers.Embedding(210, 32)(x_input[:, 2, :])
    
    x_lstm = tf.keras.layers.LSTM(
        32, return_sequences=False)(x_status + x_tm_day + x_asset)
    
    x_logits = tf.keras.layers.Dense(3)(x_lstm) 
    x_probs  = tf.nn.softmax(x_logits, axis=-1)
    return tf.keras.Model(inputs=x_input, outputs=x_probs)

In [25]:
X_train = train_data[
    ["time_index", "STATUS", "esc_index"]].values
X_valid = test_data[
    ["time_index", "STATUS", "esc_index"]].values
Y_train = train_data["next_status"].values
Y_valid = test_data["next_status"].values

lstm_model = build_model()
lstm_optim = tf.keras.optimizers.SGD(learning_rate=0.01)

lstm_model.compile(
    optimizer=lstm_optim, 
    metrics="accuracy", 
    loss='sparse_categorical_crossentropy')

lstm_model.fit(
    X_train, Y_train, epochs=15, 
    shuffle=True, batch_size=128, 
    validation_data=((X_valid, Y_valid)))

Epoch 1/15
2353/2353 [==============================] - 6s 2ms/step - loss: 0.6908 - accuracy: 0.7418 - val_loss: 0.4704 - val_accuracy: 0.9074
Epoch 2/15
2353/2353 [==============================] - 4s 2ms/step - loss: 0.1969 - accuracy: 0.9571 - val_loss: 0.3336 - val_accuracy: 0.9074
Epoch 3/15
2353/2353 [==============================] - 4s 2ms/step - loss: 0.1596 - accuracy: 0.9571 - val_loss: 0.3167 - val_accuracy: 0.9074
Epoch 4/15
2353/2353 [==============================] - 4s 2ms/step - loss: 0.1332 - accuracy: 0.9571 - val_loss: 0.2857 - val_accuracy: 0.9074
Epoch 5/15
2353/2353 [==============================] - 4s 2ms/step - loss: 0.1107 - accuracy: 0.9626 - val_loss: 0.2541 - val_accuracy: 0.9111
Epoch 6/15
2353/2353 [==============================] - 4s 2ms/step - loss: 0.0980 - accuracy: 0.9682 - val_loss: 0.2356 - val_accuracy: 0.9214
Epoch 7/15
2353/2353 [==============================] - 4s 2ms/step - loss: 0.0913 - accuracy: 0.9698 - val_loss: 0.2267 - val_accuracy:

Hyperparameter Tuning of LSTM:

Layers: 32, 64, 128

Batch Size, 32, 64, 128

Learning Rate: 0.001, 0.0001, 0.01

Epochs: 10, 15

Comparing accuracy and val_accuracy: Layers 32, batch size 128, learning rate 0.01 and epochs 15 provides the best accuracy of 97.69% and val_accuracy of 95.06%.

In [26]:
# Predict test data
predictions_lstm = lstm_model.predict(X_valid)
pred_status_lstm = tf.argmax(predictions_lstm, axis=1)

92/92 [==============================] - 0s 1ms/step


In [27]:
# Identify anomalies indexes, get number of anomalies
y_test = test_data['next_status'].values
confidence_level = 0.95
anomalies_indexes_lstm = calcualte_loss(predictions_lstm,pred_status_lstm,y_test,confidence_level )
results_lstm = test_data.copy(deep=True)
results_lstm["pred_status"] = pred_status_lstm
results_lstm.iloc[anomalies_indexes_lstm].shape

(28, 9)

In [28]:
# Drop cols, decoding, show anomalies (compare next_status with pred_status)
results_lstm = results_lstm.drop(columns=["time_index","esc_index"])

index_to_status = {value: key for key, value in status_to_index.items()}
results_lstm["STATUS"] = [index_to_status[x] for x in results_lstm["STATUS"]]
results_lstm["next_status"] = [index_to_status[x] for x in results_lstm["next_status"]]
results_lstm["pred_status"] = [index_to_status[x] for x in results_lstm["pred_status"]]

results_lstm.iloc[anomalies_indexes_lstm].head()

,dttm,ASSETNAME,STATUS,date,time,next_status,pred_status
35,2023-12-01 17:30:00,L&E/BGK/B1/E01,UP,2023-12-01,17:30:00,STOP,UP
161,2023-12-01 14:00:00,L&E/BGK/B1/E04,UP,2023-12-01,14:00:00,STOP,UP
189,2023-12-01 06:30:00,L&E/BGK/B2/E01,UP,2023-12-01,06:30:00,STOP,UP
195,2023-12-01 09:30:00,L&E/BGK/B2/E01,DOWN,2023-12-01,09:30:00,UP,DOWN
439,2023-12-01 01:00:00,L&E/CNT/B1/E06,STOP,2023-12-01,01:00:00,UP,STOP


## 7. SimpleRNN

In [29]:
def build_rnn_model():
    x_input = tf.keras.Input( 
        shape=(3,1,), dtype="int32", name="x_input")
    x_status = tf.keras.layers.Embedding(3, 32)(x_input[:, 1, :])
    x_tm_day = tf.keras.layers.Embedding(48, 32)(x_input[:, 0, :])
    x_asset  = tf.keras.layers.Embedding(210, 32)(x_input[:, 2, :])
    
    x_rnn = tf.keras.layers.SimpleRNN(
        32, return_sequences=False)(x_status + x_tm_day + x_asset)
    
    x_logits = tf.keras.layers.Dense(3)(x_rnn) 
    x_probs  = tf.nn.softmax(x_logits, axis=-1)
    return tf.keras.Model(inputs=x_input, outputs=x_probs)

In [30]:
X_train = train_data[
    ["time_index", "STATUS", "esc_index"]].values
X_valid = test_data[
    ["time_index", "STATUS", "esc_index"]].values
Y_train = train_data["next_status"].values
Y_valid = test_data["next_status"].values

rnn_model = build_rnn_model()
rnn_optim = tf.keras.optimizers.SGD(learning_rate=0.01)

rnn_model.compile(
    optimizer=rnn_optim, 
    metrics="accuracy", 
    loss='sparse_categorical_crossentropy')

rnn_model.fit(
    X_train, Y_train, epochs=15, 
    shuffle=True, batch_size=128, 
    validation_data=((X_valid, Y_valid)))

Epoch 1/15
2353/2353 [==============================] - 4s 1ms/step - loss: 0.2338 - accuracy: 0.9394 - val_loss: 0.2798 - val_accuracy: 0.9074
Epoch 2/15
2353/2353 [==============================] - 4s 1ms/step - loss: 0.1001 - accuracy: 0.9650 - val_loss: 0.2307 - val_accuracy: 0.9228
Epoch 3/15
2353/2353 [==============================] - 3s 1ms/step - loss: 0.0866 - accuracy: 0.9713 - val_loss: 0.2197 - val_accuracy: 0.9369
Epoch 4/15
2353/2353 [==============================] - 3s 1ms/step - loss: 0.0831 - accuracy: 0.9742 - val_loss: 0.2147 - val_accuracy: 0.9437
Epoch 5/15
2353/2353 [==============================] - 3s 1ms/step - loss: 0.0813 - accuracy: 0.9755 - val_loss: 0.2115 - val_accuracy: 0.9427
Epoch 6/15
2353/2353 [==============================] - 3s 1ms/step - loss: 0.0799 - accuracy: 0.9760 - val_loss: 0.2087 - val_accuracy: 0.9465
Epoch 7/15
2353/2353 [==============================] - 3s 1ms/step - loss: 0.0788 - accuracy: 0.9764 - val_loss: 0.2059 - val_accuracy:

Hyperparmeter Tunning of SimpleRNN:

Layers: 32, 64, 128

Batch Size: 32, 64, 128

Learning Rate: 0.001, 0.0001, 0.01

Epochs: 10, 15

Comparing accuracy and val_accuracy: Layers 32, batch size 128, learning rate 0.01 and epochs 15 provides the best accuracy of 97.69% and val_accuracy of 95.03%.

In [31]:
# Predict test data. #
predictions_rnn = rnn_model.predict(X_valid)
pred_status_rnn = tf.argmax(predictions_rnn, axis=1)

92/92 [==============================] - 0s 1ms/step


In [32]:
# Identify anomalies indexes, get number of anomalies
y_test = test_data['next_status'].values
confidence_level = 0.95
anomalies_indexes_rnn = calcualte_loss(predictions_rnn,pred_status_rnn,y_test,confidence_level )
results_rnn = test_data.copy(deep=True)
results_rnn["pred_status"] = pred_status_rnn
results_rnn.iloc[anomalies_indexes_rnn].shape

(29, 9)

In [33]:
# Drop cols, decoding, show anomalies (compare next_status with pred_status)
results_rnn = results_rnn.drop(columns=["time_index","esc_index"])

index_to_status = {value: key for key, value in status_to_index.items()}
results_rnn["STATUS"] = [index_to_status[x] for x in results_rnn["STATUS"]]
results_rnn["next_status"] = [index_to_status[x] for x in results_rnn["next_status"]]
results_rnn["pred_status"] = [index_to_status[x] for x in results_rnn["pred_status"]]

results_rnn.iloc[anomalies_indexes_rnn].head()

,dttm,ASSETNAME,STATUS,date,time,next_status,pred_status
6,2023-12-01 03:00:00,L&E/BGK/B1/E01,STOP,2023-12-01,03:00:00,UP,STOP
35,2023-12-01 17:30:00,L&E/BGK/B1/E01,UP,2023-12-01,17:30:00,STOP,UP
51,2023-12-01 03:00:00,L&E/BGK/B1/E02,STOP,2023-12-01,03:00:00,DOWN,STOP
161,2023-12-01 14:00:00,L&E/BGK/B1/E04,UP,2023-12-01,14:00:00,STOP,UP
189,2023-12-01 06:30:00,L&E/BGK/B2/E01,UP,2023-12-01,06:30:00,STOP,UP


## 8. Results

LSTM Performance: accuracy: 97.69%, val_accuracy: 95.06%

SimpleRNN Performance: accuracy: 97.69%, val_accuracy: 95.03%

Best Model: LSTM as LSTM val_accuracy is higher than SimpleRNN val_accuracy

In [34]:
# Save file
results_lstm.to_csv('Anomalies_lstm.csv')

In [35]:
#save model
lstm_model.save('lstm_model.h5')

In [36]:
# #load model and predict
# loaded_model_lstm = load_model('lstm_model.h5')
# predictions_lstm = loaded_model_lstm.predict(X_valid)
# pred_status_lstm = tf.argmax(predictions_lstm, axis=1)
# pred_status_lstm